<a href="https://colab.research.google.com/github/Vacuum1234/test/blob/main/TPU_experiment_copy_of_Aika_LSTM_music_test2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount Google Drive
from google.colab import drive # import drive from google colab

ROOT = "/content/gdrive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)
drive.mount(ROOT)           # we mount the google drive at /content/gdrive


/content/gdrive
Mounted at /content/gdrive


In [ ]:
import os

TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']

strategy = tf.contrib.tpu.TPUDistributionStrategy(tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER))
tpu_model = tf.contrib.tpu.keras_to_tpu_model(model, strategy=strategy)

tpu_model.summary()

tpu_number = 8
batch_size=128 * tpu_number

history = tpu_model.fit(X_tr, Y_tr,
                        epochs=100,
                        batch_size=batch_size,
                        validation_split=0.2)
tpu_model.evaluate(X_val, Y_val, batch_size=batch_size)
tpu_model.save_weights('./our_tpu_model.h5')

In [ ]:
def create_network1(network_input, n_vocab): #original
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

Ref: \\
https://towardsdatascience.com/how-to-generate-music-using-a-lstm-neural-network-in-keras-68786834d4c5

TPU: https://matthewmcateer.me/blog/using-tpus-in-google-colab/

In [ ]:
#change to the desired destination directory
#%cd /content/gdrive/MyDrive/Github-Clones/

#clone the github, only need to do this once to create the subdirectory
#!git clone https://github.com/Skuldur/Classical-Piano-Composer

Cloning into 'Classical-Piano-Composer'...
remote: Enumerating objects: 334, done.
remote: Total 334 (delta 0), reused 0 (delta 0), pack-reused 334
Receiving objects: 100% (334/334), 721.79 MiB | 32.21 MiB/s, done.
Resolving deltas: 100% (41/41), done.


In [ ]:
%cd /content/gdrive/MyDrive/Github-Clones/Classical-Piano-Composer/

/content/gdrive/MyDrive/Github-Clones/Classical-Piano-Composer


In [ ]:
ls

gdrive/  sample_data/


In [ ]:

import glob
import pickle
import numpy
from music21 import converter, instrument, note, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
#from keras.utils import np_utils
from tensorflow.keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint

# Train the LSTM

In [ ]:
#for original google acct
def train_network(weights_path, network_choice):
    """ Train a Neural Network to generate music """
    notes = get_notes()

    # get amount of pitch names
    n_vocab = len(set(notes))
    print('n_vocab =', n_vocab)
    network_input, network_output = prepare_sequences(notes, n_vocab)

    if network_choice == 1:
      model = create_network1(network_input, n_vocab)
    if network_choice == 2:
      model = create_network2(network_input, n_vocab)
    if network_choice == 3:
      model = create_network3(network_input, n_vocab)
    if network_choice == 4:
      model = create_network4(network_input, n_vocab)
    if network_choice == 5:
      model = create_network5(network_input, n_vocab)
    train(model, network_input, network_output, weights_path)

def get_notes():
    """ Get all the notes and chords from the midi files in the ./midi_songs directory """
    notes = []

    for file in glob.glob("/content/gdrive/MyDrive/Github-Clones/Classical-Piano-Composer/midi_songs/*"):

        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse()
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    with open('/content/gdrive/MyDrive/Github-Clones/Classical-Piano-Composer/data/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)

    return notes

def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100
    ndrops = 5
    a1 = (sequence_length - ndrops)//2
    a2 = a1 + 1
    a3 = a1 + ndrops - 1
    a4 = a3 + 1

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length - ndrops, 1):
      sequence_in = notes[i:i + sequence_length]
      sequence_out = notes[i + sequence_length]
 #       sequence_in = notes[i:i + (sequence_length-ndrops)/2]
 #       sequence_out = notes[i + sequence_length/2:i + sequence_length/2+ndrops]
      network_input.append([note_to_int[char] for char in sequence_in])
      network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))

    # normalize the input
    network_input = network_input / float(n_vocab)

    network_output = to_categorical(network_output)

    return (network_input, network_output)

def create_network1(network_input, n_vocab): #original
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model

def create_network2(network_input, n_vocab): #changed dropout to 0.1
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.1,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.1,))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.1))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.1))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model

def create_network3(network_input, n_vocab): #changed activation function to tanh
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('tanh'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('tanh'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model

def create_network4(network_input, n_vocab): #changed lstm nodes to 256
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(256, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(256))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model

def create_network5(network_input, n_vocab): #added another lstm layer
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(512, return_sequences=True))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model

def train(model, network_input, network_output, weights_path):
    """ train the neural network """
    filepath = weights_path + "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    #model.fit(network_input, network_output, epochs=2, batch_size=128, callbacks=callbacks_list)
    model.fit(network_input, network_output, epochs=150, batch_size=128, callbacks=callbacks_list)

In [ ]:
#for test acct

def train_network(weights_path, network_choice):
    """ Train a Neural Network to generate music """
    notes = get_notes()

    # get amount of pitch names
    n_vocab = len(set(notes))
    print('n_vocab =', n_vocab)
    network_input, network_output = prepare_sequences(notes, n_vocab)

    if network_choice == 1:
      model = create_network1(network_input, n_vocab)
    if network_choice == 2:
      model = create_network2(network_input, n_vocab)
    if network_choice == 3:
      model = create_network3(network_input, n_vocab)
    if network_choice == 4:
      model = create_network4(network_input, n_vocab)
    if network_choice == 5:
      model = create_network5(network_input, n_vocab)
    train(model, network_input, network_output, weights_path)

def get_notes():
    """ Get all the notes and chords from the midi files in the ./midi_songs directory """
    notes = []

    for file in glob.glob("/content/gdrive/MyDrive/midi_songs/*"):

        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse()
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    with open('/content/gdrive/MyDrive/notes-2', 'wb') as filepath:
        pickle.dump(notes, filepath)

    return notes

def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100
    ndrops = 5
    a1 = (sequence_length - ndrops)//2
    a2 = a1 + 1
    a3 = a1 + ndrops - 1
    a4 = a3 + 1

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length - ndrops, 1):
      sequence_in = notes[i:i + sequence_length]
      sequence_out = notes[i + sequence_length]
 #       sequence_in = notes[i:i + (sequence_length-ndrops)/2]
 #       sequence_out = notes[i + sequence_length/2:i + sequence_length/2+ndrops]
      network_input.append([note_to_int[char] for char in sequence_in])
      network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))

    # normalize the input
    network_input = network_input / float(n_vocab)

    network_output = to_categorical(network_output)

    return (network_input, network_output)

def create_network1(network_input, n_vocab): #original
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model

def create_network2(network_input, n_vocab): #changed dropout to 0.1
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.1,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.1,))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.1))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.1))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model

def create_network3(network_input, n_vocab): #changed activation function to tanh
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('tanh'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('tanh'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model

def create_network4(network_input, n_vocab): #changed lstm nodes to 256
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(256, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(256))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model

def create_network5(network_input, n_vocab): #added another lstm layer
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(512, return_sequences=True))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model

def train(model, network_input, network_output, weights_path):
    """ train the neural network """
    filepath = weights_path + "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    #model.fit(network_input, network_output, epochs=2, batch_size=128, callbacks=callbacks_list)
    model.fit(network_input, network_output, epochs=150, batch_size=128, callbacks=callbacks_list)

In [ ]:
#set weights path and filename
network_choice=4
weights_path = "/content/gdrive/MyDrive/Github-Clones/Classical-Piano-Composer/weights/" + str(network_choice) + "/"
#weights_filename = "weights-improvement-01-5.1848-bigger.hdf5"
#weights_filename = "my_model_100_epoch.hdf5"
weights_filename = "weights-improvement-01-5.2259-bigger.hdf5"
weights_file = weights_path + weights_filename

generate(weights_file, network_choice)

FileNotFoundError: [Errno 2] No such file or directory: '/content/gdrive/MyDrive/Github-Clones/Classical-Piano-Composer/data/notes'

In [ ]:
#set path to where weights will be stores
network_choice = 5
weights_path = "/content/gdrive/MyDrive/Github-Clones/Classical-Piano-Composer/weights/" + str(network_choice) + "/"
train_network(weights_path, network_choice)

Parsing /content/gdrive/MyDrive/Github-Clones/Classical-Piano-Composer/midi_songs/relmstheme-piano.mid
Parsing /content/gdrive/MyDrive/Github-Clones/Classical-Piano-Composer/midi_songs/rufus.mid
Parsing /content/gdrive/MyDrive/Github-Clones/Classical-Piano-Composer/midi_songs/Eternal_Harvest.mid
Parsing /content/gdrive/MyDrive/Github-Clones/Classical-Piano-Composer/midi_songs/FF4.mid
Parsing /content/gdrive/MyDrive/Github-Clones/Classical-Piano-Composer/midi_songs/Ff7-Cinco.mid
Parsing /content/gdrive/MyDrive/Github-Clones/Classical-Piano-Composer/midi_songs/EyesOnMePiano.mid
Parsing /content/gdrive/MyDrive/Github-Clones/Classical-Piano-Composer/midi_songs/Life_Stream.mid
Parsing /content/gdrive/MyDrive/Github-Clones/Classical-Piano-Composer/midi_songs/FF3_Battle_(Piano).mid
Parsing /content/gdrive/MyDrive/Github-Clones/Classical-Piano-Composer/midi_songs/tpirtsd-piano.mid
Parsing /content/gdrive/MyDrive/Github-Clones/Classical-Piano-Composer/midi_songs/dayafter.mid
Parsing /content/gd

Epoch 1/150
316/359 [=========================>....] - ETA: 26s - loss: 4.9495

# Generate Music

In [ ]:
""" This module generates notes for a midi file using the
    trained neural network """
import pickle
import numpy
from music21 import instrument, note, stream, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import BatchNormalization as BatchNorm
from keras.layers import Activation

def generate(weights_file, network_choice):
    """ Generate a piano midi file """
    #load the notes used to train the model
    #with open('/content/gdrive/MyDrive/Github-Clones/Classical-Piano-Composer/data/notes', 'rb') as filepath:
        #notes = pickle.load(filepath)
    notes=get_notes()
    n_vocab=326

    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))
    # Get all pitch names

    network_input, normalized_input = prepare_sequences1(notes, pitchnames, n_vocab)

    if network_choice==1:
      model = create_network1(normalized_input, n_vocab)
    if network_choice == 2:
      model = create_network2(normalized_input, n_vocab)
    if network_choice == 3:
      model = create_network3(normalized_input, n_vocab)
    if network_choice == 4:
      model = create_network4(normalized_input, n_vocab)
    if network_choice == 5:
      model = create_network5(normalized_input, n_vocab)
    model = load_weights(model, weights_file)
    prediction_output = generate_notes(model, network_input, pitchnames, n_vocab)
    create_midi(prediction_output)

def prepare_sequences1(notes, pitchnames, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    # map between notes and integers and back
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    sequence_length = 100
    network_input = []
    output = []
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    normalized_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    normalized_input = normalized_input / float(n_vocab)

    return (network_input, normalized_input)

def load_weights(model, weights_file):

    # Load the weights to each node
    model.load_weights(weights_file)
    return model

def generate_notes(model, network_input, pitchnames, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction
    start = numpy.random.randint(0, len(network_input)-1)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    prediction_output = []

    # generate 500 notes
    for note_index in range(500):
        prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = numpy.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)

        pattern.append(index)
        pattern = pattern[1:len(pattern)]

    return prediction_output

def create_midi(prediction_output):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='test_output'+str(network_choice)+"-"+weights_filename[20:23]+".mid")


    #filename has network choice then epoch #

In [ ]:
#set weights path and filename
network_choice=2
weights_path = "/content/gdrive/MyDrive/"
#weights_filename = "weights-improvement-01-5.1848-bigger.hdf5"
#weights_filename = "my_model_100_epoch.hdf5"
weights_filename = "weights-improvement-139-0.1123-bigger.hdf5"
weights_file = weights_path + weights_filename



generate(weights_file, network_choice)

Parsing /content/gdrive/MyDrive/midi_songs/cosmo.mid
Parsing /content/gdrive/MyDrive/midi_songs/Fiend_Battle_(Piano).mid
Parsing /content/gdrive/MyDrive/midi_songs/JENOVA.mid
Parsing /content/gdrive/MyDrive/midi_songs/thenightmarebegins.mid
Parsing /content/gdrive/MyDrive/midi_songs/FF4.mid
Parsing /content/gdrive/MyDrive/midi_songs/ff4-fight1.mid
Parsing /content/gdrive/MyDrive/midi_songs/Life_Stream.mid
Parsing /content/gdrive/MyDrive/midi_songs/rufus.mid
Parsing /content/gdrive/MyDrive/midi_songs/Zelda_Overworld.mid
Parsing /content/gdrive/MyDrive/midi_songs/figaro.mid
Parsing /content/gdrive/MyDrive/midi_songs/ff8-lfp.mid
Parsing /content/gdrive/MyDrive/midi_songs/BlueStone_LastDungeon.mid
Parsing /content/gdrive/MyDrive/midi_songs/0fithos.mid
Parsing /content/gdrive/MyDrive/midi_songs/dontbeafraid.mid
Parsing /content/gdrive/MyDrive/midi_songs/FFIX_Piano.mid
Parsing /content/gdrive/MyDrive/midi_songs/ff4_piano_collections-main_theme.mid
Parsing /content/gdrive/MyDrive/midi_songs/d

NameError: name 'model' is not defined

In [ ]:
!pip install selenium
!pip install chromedriver-py
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

options = webdriver.ChromeOptions()
options.add_argument("start-maximized")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(options=options, executable_path=r'C:\WebDrivers\chromedriver.exe')
driver.get("https://www.inipec.gov.it/cerca-pec/-/pecs/companies")
WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.CSS_SELECTOR,"iframe[name^='a-'][src^='https://www.google.com/recaptcha/api2/anchor?']")))
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//span[@id='recaptcha-anchor']"))).click()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.2/460.2 kB 41.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 MB 23.9 MB/s eta 0:00:00


TypeError: WebDriver.__init__() got an unexpected keyword argument 'executable_path'